# Genrate bacteria counts for the manuscript

## Read data

In [1]:
import os
import pandas as pd

from pathlib import Path


datamatrixDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240531/data/data_matrix.csv')
infoDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'saur_cohort_20250113', 'source', 'patient_tube_id_mapping_full.tsv'), sep='\t')

datamatrixDf.shape, infoDf.shape

((116754, 194), (3117, 18))

In [3]:
dataDf = datamatrixDf[['person_id']].drop_duplicates()
dataDf['Flag'] = True
mergedDf = infoDf.merge(
    dataDf,
    how='left',
    left_on=['PATIENT_ID'],
    right_on=['person_id']
)
mergedDf.Flag = mergedDf.Flag.fillna(False)
mergedDf

/tmp/ipykernel_742971/2471855643.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mergedDf.Flag = mergedDf.Flag.fillna(False)


,tube_code,PATIENT_ID,db_ID,pt_age,pt_gender,EPISODE_ID,hospital_admission,hospital_discharge,hospital_in_last_year,genome_species,species_reported,contig_number,length,tube_code_duplicate_or_old,date_of_collection,ID_number,location_additional_02,collected_from_original,person_id,Flag
0,AH19J072,213972.0,3JNQS,64,Male,12931474.0,2019-10-17,2019-11-29,yes,Escherichia coli,Escherichia coli,92,5131021,NaN,2019-10-30,19-303-0997,A-7EA;HAEM,Blood,NaN,False
1,AH19J074,213972.0,3JNQS,64,Male,12931474.0,2019-10-17,2019-11-29,yes,Streptococcus sp. D19,Streptococcus mitis group,84,2022991,NaN,2019-10-30,19-303-1174,A-7EA;HAEM,Blood Peripheral,NaN,False
2,AH19B003,526238.0,567AE,85,Male,640261.0,2019-02-04,2019-02-07,no,Escherichia coli,Escherichia coli,93,4942915,NaN,2019-02-03,19-034-1596,A-4WB;GMC,Blood,526238.0,True
3,AH20A024,788941.0,3YBDS,58,Female,13194824.0,2020-01-16,2020-01-19,no,Escherichia coli,Escherichia coli,110,5149157,NaN,2020-01-16,20-016-2112,S-ED;Emergency,Blood Venous,788941.0,True
4,AH20L041,1023735.0,2VJ78,42,Male,14210147.0,2020-12-18,2021-01-05,yes,Staphylococcus aureus,Staphylococcus aureus,46,2735216,NaN,2020-12-16,20-351-2856,Emergency Dept;nan,Blood,1023735.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,ALF22L076,639020.0,3S6HL,84,Male,17219899.0,2022-12-15,2022-12-20,yes,Streptococcus vestibularis,Streptococcus salivarius group,107,1938627,NaN,2022-12-15,22-349-2150,Ward 4WB,Blood Peripheral,639020.0,True
3113,ALF22L089,655404.0,6EWBM,67,Female,NaN,NaN,NaN,NaN,Proteus columbae,Proteus vulgaris,152,3972219,NaN,2022-12-17,22-351-1465,Comm/Amb Clinic,Blood,655404.0,True
3114,ALF22L138,2572048.0,YM5PJ,61,Male,NaN,NaN,NaN,NaN,Enterobacter roggenkampii,Enterobacter cloacae complex,66,4876179,NaN,2022-12-27,22-361-0202,Ward 7 East,Hick White Lum,NaN,False
3115,ALF23A102,2054964.0,5DYAH,35,Female,17294287.0,2023-01-12,2023-02-01,yes,Streptococcus constellatus,Streptococcus anginosus group,54,1869828,NaN,2023-01-11,23-011-3305,ICU,Blood Venous,2054964.0,True


In [42]:
countsDf = mergedDf.genome_species.value_counts().reset_index().rename(columns={'count': 'Total Count'}).merge(
    mergedDf[mergedDf.Flag].genome_species.value_counts().reset_index().rename(columns={'count': 'Study Count'}),
    how='left',
    on=['genome_species']
)
countsDf

,genome_species,Total Count,Study Count
0,Escherichia coli,850,800.0
1,Staphylococcus aureus,557,530.0
2,Enterococcus faecium,220,213.0
3,Klebsiella pneumoniae,199,185.0
4,Enterococcus faecalis,151,139.0
...,...,...,...
93,Acinetobacter seifertii,1,1.0
94,Acinetobacter oleivorans,1,1.0
95,Acinetobacter variabilis,1,1.0
96,Proteus columbae,1,1.0


## Gram negatives

### Acinetobacter species

In [43]:
countsDf[countsDf.genome_species.str.contains('Acinetobacter')]

,genome_species,Total Count,Study Count
33,Acinetobacter nosocomialis,9,9.0
35,Acinetobacter baumannii,8,7.0
37,Acinetobacter pittii,8,7.0
52,Acinetobacter lwoffii,4,4.0
55,Acinetobacter ursingii,4,4.0
62,Acinetobacter radioresistens,3,3.0
84,Acinetobacter courvalinii,1,1.0
93,Acinetobacter seifertii,1,1.0
94,Acinetobacter oleivorans,1,1.0
95,Acinetobacter variabilis,1,1.0


In [47]:
countsDf[countsDf.genome_species.str.contains('Acinetobacter')][['Total Count', 'Study Count']].sum()

Total Count    41.0
Study Count    39.0
dtype: float64

### Burkholderia cepacia complex

In [48]:
countsDf[countsDf.genome_species.str.contains('Burkholderia')]

,genome_species,Total Count,Study Count
71,Burkholderia stabilis,2,2.0
74,Burkholderia multivorans,1,1.0
85,Burkholderia cenocepacia,1,1.0
89,Burkholderia contaminans,1,1.0


In [49]:
countsDf[countsDf.genome_species.str.contains('Burkholderia')][['Total Count', 'Study Count']].sum()

Total Count    5.0
Study Count    5.0
dtype: float64

### Pseudomonas aeruginosa

In [52]:
countsDf[countsDf.genome_species.str.contains('Pseudomonas aeruginosa')][['Total Count', 'Study Count']].sum()

Total Count    149.0
Study Count    145.0
dtype: float64

### Stenotrophomonas maltophilia

In [51]:
countsDf[countsDf.genome_species.str.contains('Stenotrophomonas maltophilia')][['Total Count', 'Study Count']].sum()

Total Count    27.0
Study Count    27.0
dtype: float64

### Citrobacter freundii complex

In [53]:
countsDf[countsDf.genome_species.str.contains('Citrobacter freundii')][['Total Count', 'Study Count']].sum()

Total Count    4.0
Study Count    4.0
dtype: float64

### Citobacter koseri

In [54]:
countsDf[countsDf.genome_species.str.contains('Citrobacter koseri')][['Total Count', 'Study Count']].sum()

Total Count    14.0
Study Count    11.0
dtype: float64

### Enterobacter cloacae complex

In [55]:
countsDf[countsDf.genome_species.str.contains('Enterobacter cloacae')][['Total Count', 'Study Count']].sum()

Total Count    8.0
Study Count    8.0
dtype: float64

### Escherichia coli

In [56]:
countsDf[countsDf.genome_species.str.contains('Escherichia coli')][['Total Count', 'Study Count']].sum()

Total Count    850.0
Study Count    800.0
dtype: float64

### Klebsiella aerogenes

In [57]:
countsDf[countsDf.genome_species.str.contains('Klebsiella aerogenes')][['Total Count', 'Study Count']].sum()

Total Count    32.0
Study Count    31.0
dtype: float64

### Klebsiella oxytoca

In [58]:
countsDf[countsDf.genome_species.str.contains('Klebsiella oxytoca')][['Total Count', 'Study Count']].sum()

Total Count    23.0
Study Count    20.0
dtype: float64

### Kiebsiella pneumoniae

In [59]:
countsDf[countsDf.genome_species.str.contains('Klebsiella pneumoniae')][['Total Count', 'Study Count']].sum()

Total Count    199.0
Study Count    185.0
dtype: float64

### Proteus mirabilis

In [60]:
countsDf[countsDf.genome_species.str.contains('Proteus mirabilis')][['Total Count', 'Study Count']].sum()

Total Count    58.0
Study Count    52.0
dtype: float64

### Raoultella ornithinolytica

In [61]:
countsDf[countsDf.genome_species.str.contains('Raoultella ornithinolytica')][['Total Count', 'Study Count']].sum()

Total Count    6.0
Study Count    5.0
dtype: float64

### Raoultella planticola

In [62]:
countsDf[countsDf.genome_species.str.contains('Raoultella planticola')][['Total Count', 'Study Count']].sum()

Total Count    4.0
Study Count    4.0
dtype: float64

### Serratia marcescens

In [63]:
countsDf[countsDf.genome_species.str.contains('Serratia marcescens')][['Total Count', 'Study Count']].sum()

Total Count    40.0
Study Count    39.0
dtype: float64

## Gram positives

### Staphylococcus aureus

In [65]:
countsDf[countsDf.genome_species.str.contains('Staphylococcus aureus')][['Total Count', 'Study Count']].sum()

Total Count    557.0
Study Count    530.0
dtype: float64

### Enterococcus faecalis

In [66]:
countsDf[countsDf.genome_species.str.contains('Enterococcus faecalis')][['Total Count', 'Study Count']].sum()

Total Count    151.0
Study Count    139.0
dtype: float64

### Enterococcus faecium

In [67]:
countsDf[countsDf.genome_species.str.contains('Enterococcus faecium')][['Total Count', 'Study Count']].sum()

Total Count    220.0
Study Count    213.0
dtype: float64

### Enterococcus Other

In [69]:
countsDf[countsDf.genome_species.str.contains('Enterococcus')]

,genome_species,Total Count,Study Count
2,Enterococcus faecium,220,213.0
4,Enterococcus faecalis,151,139.0
44,Enterococcus gallinarum,5,4.0
58,Enterococcus casseliflavus,3,2.0
68,Enterococcus raffinosus,2,NaN
78,Enterococcus avium,1,1.0
87,Enterococcus durans,1,1.0


### Streptococcus pyogenes

In [70]:
countsDf[countsDf.genome_species.str.contains('Streptococcus pyogenes')][['Total Count', 'Study Count']].sum()

Total Count    37.0
Study Count    35.0
dtype: float64

### Steptococcus agalactiae

In [71]:
countsDf[countsDf.genome_species.str.contains('Streptococcus agalactiae')][['Total Count', 'Study Count']].sum()

Total Count    57.0
Study Count    52.0
dtype: float64

### Streptococcus dysgalactiae

In [72]:
countsDf[countsDf.genome_species.str.contains('Streptococcus dysgalactiae')][['Total Count', 'Study Count']].sum()

Total Count    58.0
Study Count    56.0
dtype: float64

### streptococcus mitis group

In [73]:
countsDf[countsDf.genome_species.str.contains('mitis')][['Total Count', 'Study Count']].sum()

Total Count    35.0
Study Count    32.0
dtype: float64

### Streptococcus salivarius group

In [74]:
countsDf[countsDf.genome_species.str.contains('Streptococcus salivarius')][['Total Count', 'Study Count']].sum()

Total Count    41.0
Study Count    38.0
dtype: float64

### Streptococcus anginosus group

In [75]:
countsDf[countsDf.genome_species.str.contains('Streptococcus anginosus')][['Total Count', 'Study Count']].sum()

Total Count    28.0
Study Count    28.0
dtype: float64

### Streptococcus bovis group

In [76]:
countsDf[countsDf.genome_species.str.contains('Streptococcus bovis')][['Total Count', 'Study Count']].sum()

Total Count    0.0
Study Count    0.0
dtype: float64

### Streptococcus mutans group

In [77]:
countsDf[countsDf.genome_species.str.contains('Streptococcus mutans')][['Total Count', 'Study Count']].sum()

Total Count    3.0
Study Count    3.0
dtype: float64

### Streptococcus sanguinis group

In [78]:
countsDf[countsDf.genome_species.str.contains('Streptococcus sanguinis')][['Total Count', 'Study Count']].sum()

Total Count    22.0
Study Count    22.0
dtype: float64

### Streptococcus pneumoniae

In [79]:
countsDf[countsDf.genome_species.str.contains('Streptococcus pneumoniae')][['Total Count', 'Study Count']].sum()

Total Count    40.0
Study Count    36.0
dtype: float64